# **Redes Bayesianas**

Supongamos una situación en la que tenemos tres variables: **Caries**, **Dolor** y **Huecos**. Por nuestro propio conocimiento, podemos inferir que la caries es la causante del dolor y/o de los huecos en los dientes. Por ese motivo representaremos la red bayesiana con tres nodos, donde el nodo **Caries** es padre de los nodos **Dolor** y **Huecos**. Representaremos en sus respectivas tablas sus valores probabilísticos.

<img src="imgs/red_bayesiana.JPG" width=50%>

Utilizaremos la librería [*pomegranate*](https://pomegranate.readthedocs.io/en/latest/index.html) para crear la red y hacer inferencia sobre ella.

In [ ]:
%matplotlib inline
from pomegranate import *

caries = DiscreteDistribution({'Yes': 0.8, 'No': 0.2})

dolor = ConditionalProbabilityTable(
        [['Yes', 'True', 0.6],
         ['Yes', 'False', 0.4],
         ['No', 'True', 0.1],
         ['No', 'False', 0.9]],
         [caries])

huecos = ConditionalProbabilityTable(
        [['Yes', 'True', 0.9],
         ['Yes', 'False', 0.1],
         ['No', 'True', 0.2],
         ['No', 'False', 0.8]],
         [caries])

s1 = Node(caries, name="caries")
s2 = Node(dolor, name="dolor")
s3 = Node(huecos, name="huecos")

model = BayesianNetwork("Dentista")
model.add_states(s1, s2, s3)
model.add_edge(s1, s2)
model.add_edge(s1, s3)

model.bake()
# model.plot()

Calcularemos ahora qué probabilidad de padecer caries tenemos si nos duele una muela.

In [ ]:

# observations = { }
# observations = { 'caries' : 'Yes'}
observations = { 'dolor' : 'True', 'huecos': 'True'}

beliefs = model.predict_proba(observations)

for state, belief in zip( model.states, beliefs ):
    if type(belief) is not str:
        print(state.name)
        print("Estado", belief.parameters)

#### Ejercicio 1

Calcula a mano y mediante la librería los siguientes casos:

- Probabilidad de padecer caries tenemos si tenemos huecos.
- Probabilidad de padecer caries.
- Probabilidad de no padecer caries tenemos si tenemos huecos y dolor.
- Probabilidad de tener huecos.

Verifica que lo que calculas a mano coincide con las respuestas de la librería.

#### Ejercicio 2

Implementa mediante la librería la siguiente red:

<img src="imgs/red_bayesiana_ejemplo_2.JPG" width=50%>

Calcula los valores de las diferentes variables si se produce un robo.

In [ ]:
%matplotlib inline
from pomegranate import *

robo = DiscreteDistribution({'Yes_robo': 0.001, 'No_robo': 0.999})
terremoto = DiscreteDistribution({'Yes_terremoto': 0.002, 'No_terremoto': 0.998})

alarma = ConditionalProbabilityTable(
        [['Yes_robo', 'Yes_terremoto', 'Salta_alarma', 0.95],
         ['Yes_robo', 'Yes_terremoto', 'No_salta_alarma', 0.05],
         ['Yes_robo', 'No_terremoto', 'Salta_alarma', 0.94],
         ['Yes_robo', 'No_terremoto', 'No_salta_alarma', 0.06],
         ['No_robo', 'Yes_terremoto', 'Salta_alarma', 0.29],
         ['No_robo', 'Yes_terremoto', 'No_salta_alarma', 0.71],
         ['No_robo', 'No_terremoto', 'Salta_alarma', 0.001],
         ['No_robo', 'No_terremoto', 'No_salta_alarma', 0.999]],
         [robo, terremoto])

juan = ConditionalProbabilityTable(
        [['Salta_alarma','Juan_llama', 0.90],
         ['Salta_alarma','Juan_no_llama', 0.10],
         ['No_salta_alarma','Juan_llama', 0.05],
         ['No_salta_alarma','Juan_no_llama', 0.95]],
         [alarma])

maria = ConditionalProbabilityTable(
        [['Salta_alarma','Maria_llama', 0.70],
         ['Salta_alarma','Maria_no_llama', 0.30],
         ['No_salta_alarma','Maria_llama', 0.01],
         ['No_salta_alarma','Maria_no_llama', 0.99]],
         [alarma])

s1 = Node(robo, name="Robo")
s2 = Node(terremoto, name="Terremoto")
s3 = Node(alarma, name="Alarma")
s4 = Node(juan, name="Juan")
s5 = Node(maria, name="Maria")


model = BayesianNetwork("Avisos")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.add_edge(s3, s4)
model.add_edge(s3, s5)

model.bake()
#model.plot()

In [ ]:
# observations = {}
observations = {'Robo':'Yes_robo', 'Juan':'Juan_llama'}
# observations = {'Robo':'Yes_robo', 'Juan':'Juan_llama', 'Terremoto':'Yes_terremoto','Maria':'Maria_llama'}
# observations = {'Juan':'Juan_llama', 'Maria':'Maria_llama'}
# observations = {'Alarma' : 'Salta_alarma', 'Juan':'Juan_llama', 'Maria':'Maria_llama'}
# observations = {'Alarma' : 'Salta_alarma', 'Juan':'Juan_llama'}

beliefs = model.predict_proba(observations)

for state, belief in zip( model.states, beliefs ):
    if type(belief) is not str:
        print(state.name)
        print("Estado", belief.parameters)

## **Redes bayesianas a partir de datos**

En un caso normal no contaremos con el DAG (directed acyclid graph) ni con las CPT (conditional probability tables). Tendremos que construir todo eso a partir de los datos que podamos recoger.

El siguiente *script* nos generará un *dataframe* con 100.000 muestras con el que reconstruiremos el ejemplo anterior.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(666)

earthquake = np.random.choice(2, 100000, p=[0.998, 0.002])
robbery = np.random.choice(2, 100000, p=[0.999, 0.001])

alarm = []

for e,r in zip(earthquake, robbery):
    if e==0 and r==0:
        a = np.random.choice(2, p=[0.999, 0.001])
    if e==0 and r==1:
        a = np.random.choice(2, p=[0.06, 0.94])
    if e==1 and r==0:
        a = np.random.choice(2, p=[0.71, 0.29])
    if e==1 and r==1:
        a = np.random.choice(2, p=[0.05, 0.95])
    alarm.append(a)
        
juan = []
maria = []
        
for a in alarm:
    if a==0:
        j = np.random.choice(2, p=[0.95, 0.05])
        m = np.random.choice(2, p=[0.99, 0.01])
    if a==1:
        j = np.random.choice(2, p=[0.10, 0.90])
        m = np.random.choice(2, p=[0.30, 0.70])
    juan.append(j)
    maria.append(m)

aux = np.stack((np.array(earthquake), np.array(robbery), np.array(alarm), np.array(juan), np.array(maria)), axis=0)
samples = np.transpose(aux)

df = pd.DataFrame(data=samples, columns=["E", "R", "A", "J", "M"])

df

Fíjate que, si no contáramos con suficientes datos, las probabilidades que calculáramos podrían no ser muy precisas.

In [ ]:
print("Porcentaje de veces que hay un robo:", np.round( sum(df['R'])/df.shape[0],4) )

In [ ]:
print("Porcentaje de veces que hay un terremoto:", np.round( sum(df['E'])/df.shape[0],4) )

In [ ]:
print("Porcentaje de veces que Juan llama cuando suena la alarma:", np.round( sum(df['A'] * df['J']) / sum(df['A']),2) )

In [ ]:
print("Porcentaje de veces que María llama cuando suena la alarma:", np.round( sum(df['A'] * df['M']) / sum(df['A']),2) )

### Creación del DAG

Una vez obtenido el *dataset*, el siguiente paso será crear el DAG. Utilizaremos la librería [bnlearn](https://erdogant.github.io/bnlearn/pages/html/index.html). Para ello haremos:

In [ ]:
import bnlearn as bn

DAG = bn.structure_learning.fit(df, methodtype='ex', scoretype='bic')

In [ ]:
# print adjacency matrix
print(DAG['adjmat'])

In [ ]:
# bn.plot(DAG, interactive=True, params_interactive={'notebook':True})
bn.plot(DAG)

Por supuesto, es posible también crear el DAG a mano.

In [ ]:
# Import the library
import bnlearn

# Define the network structure
edges = [('Cloudy', 'Sprinkler'),
         ('Cloudy', 'Rain'),
         ('Sprinkler', 'Wet_Grass'),
         ('Rain', 'Wet_Grass')]

# Make the actual Bayesian DAG
DAG_water = bnlearn.make_DAG(edges)

bnlearn.plot(DAG_water)

### Aprendizaje de los parámetros

A partir del grafo y de los datos es posible crear las tablas de probabilidad condicionada.

In [ ]:
model = bn.parameter_learning.fit(DAG, df, methodtype='maximumlikelihood')

### Inferencia

In [ ]:
# q1 = bn.inference.fit(model, variables=['A','M'], evidence={'R':1, 'J':1})
q1 = bn.inference.fit(model, variables=['A'], evidence={'R':0, 'E':1})
print(q1.df)

In [ ]:
q2 = bn.inference.fit(model, variables=['J','M'], evidence={'R':1, 'A':1, 'E':1})
print(q2.df)

### Almacenamiento y carga del modelo

Cuando tenemos gran cantidad de datos y muchas variables, el proceso de encontrar el DAG y calcular las CPT puede ser muy costoso. En ese caso, es necesario contar con funciones para almacenar y recuperar nuestra red bayesiana.

In [ ]:
# Save model
bn.save(model, filepath='bnlearn_model', overwrite=True)

# Load model
model = bn.load(filepath='bnlearn_model')

### Ejercicios

1) Calcula las CPTs de las variables *Earthquake* y *Alarm* y comprueba que obtienes los mismos valores que los dados por la librería <code>bn.parameter_learning.fit</code>.

In [ ]:
print(np.round( sum((1-df['R']))/df.shape[0],7) )

2) Utilizando el dataset "asia", que ya viene incluido en la librería, realiza las siguientes tareas:
- Crear su correspondiente DAG. Prueba otros valores para el parámetro <code>methodtype</code>, como **hc**.
- Calcular las CPT.
- Hacer las inferencias:
    - ¿Qué probabilidad hay de tener disnea siendo fumador?
    - ¿Qué probabilidad hay de tener una radiografía positiva siendo fumador habiendo visitado Asia?
    
La figura siguiente muestra el grafo original. Comprueba que el DAG calculado coincide con éste.

<img src="imgs/ASIA_BN.jpeg" width=40%/>

In [ ]:
import bnlearn as bn
df_asia = bn.import_example('asia', n=10000)
print(df_asia)

In [ ]:
edges = [
    ('asia', 'tub'),
    ('tub', 'either'),
    ('smoke', 'lung'),
    ('smoke', 'bronc'),
    ('lung', 'either'),
    ('either', 'xray'),
    ('either', 'dysp'),
    ('bronc', 'dysp'),
]

dag_asia = bn.make_DAG(edges)
model_asia = bn.parameter_learning.fit(dag_asia, df_asia, methodtype='maximumlikelihood')
bn.plot(dag_asia)

In [ ]:
bn.inference.fit(model_asia, variables=['dysp'], evidence={'smoke':1})

## Referencias

https://towardsdatascience.com/a-step-by-step-guide-in-detecting-causal-relationships-using-bayesian-structure-learning-in-python-c20c6b31cee5